In [556]:
import pandas as pd

In [557]:
df=pd.read_csv("/Users/vambat1/Documents/Projects/Machine-Learning/04-Movie-Recommendation-System/data/artifact/2024-01-30-14-52-05/data_ingestion/ingested_data/ingested_test_data/imdb_movies.csv")

In [558]:
df.head()

,Unnamed: 0,names,date_x,score,genre,overview,crew,orig_title,status,orig_lang,budget_x,revenue,country
0,2958,The Sorcerer's Apprentice,09/09/2010,61.0,"Fantasy, Adventure, Action",Balthazar Blake is a master sorcerer in modern...,"Nicolas Cage, Balthazar Blake, Jay Baruchel, D...",The Sorcerer's Apprentice,Released,English,160000000.0,217986320.0,AU
1,9293,With Aunt It's Not a Sin,12/23/1980,61.0,Comedy,"The rich Gloria, resident for years in the Uni...","Marina Hedman, Gloria, Antonella Antinori, Ant...",Con la zia non è peccato,Released,Italian,150600000.0,414413080.2,IT
2,6821,Being John Malkovich,12/03/1999,74.0,"Comedy, Drama, Fantasy","One day at work, unsuccessful puppeteer Craig ...","John Cusack, Craig Schwartz, John Malkovich, J...",Being John Malkovich,Released,English,13000000.0,32382381.0,AU
3,2025,Oz the Great and Powerful,03/07/2013,59.0,"Fantasy, Adventure, Family","Oscar Diggs, a small-time circus illusionist a...","James Franco, Oz, Mila Kunis, Theodora / Wicke...",Oz the Great and Powerful,Released,English,200000000.0,490359051.0,AU
4,1780,Hatching,05/26/2022,63.0,"Horror, Fantasy",12 year old Tinja is desperate to please her m...,"Siiri Solalinna, Tinja, Sophia Heikkilä, Mothe...",Pahanhautoja,Released,Finnish,3896000.0,490000.0,AU


In [559]:
df['year'] = df['date_x'].apply(lambda x: str(x).split("/")[2].split()[0])

In [560]:
df = df[['names', 'overview', 'genre']]

In [561]:
df.shape

(2036, 3)

In [562]:
# df['orig_lang']=df['orig_lang'].str.strip()
# df['orig_lang'].unique()

In [563]:
df.isna().sum()

names        0
overview     0
genre       12
dtype: int64

In [564]:
df.dtypes

names       object
overview    object
genre       object
dtype: object

In [529]:
df.dropna(inplace=True)
df=df.reset_index(drop=True)

In [530]:
unique_genres = {genre.strip().replace(" ", "_") for movie in df['genre'] for genre in movie.split(',')}
for genre in unique_genres:
    df[genre] = df['genre'].str.contains(genre).astype(int)

In [531]:
df.columns

Index(['names', 'overview', 'genre', 'TV_Movie', 'Adventure', 'Comedy',
       'Fantasy', 'Romance', 'Family', 'History', 'Documentary', 'War',
       'Crime', 'Drama', 'Horror', 'Thriller', 'Animation', 'Action',
       'Science_Fiction', 'Mystery', 'Western', 'Music'],
      dtype='object')

In [532]:
df = df.drop(columns=['genre'], axis=1)

In [533]:
df.to_csv("updated_imdb.csv", index=True)

In [534]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=df.shape[0], stop_words="english")
cv

CountVectorizer(max_features=2024, stop_words='english')

In [555]:
vector = cv.fit_transform(df['overview'].values.astype('U')).toarray()
vector.shape

(2024, 2024)

In [536]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vector)
similarity

array([[1.        , 0.        , 0.09449112, ..., 0.        , 0.        ,
        0.07905694],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.09449112, 0.        , 1.        , ..., 0.        , 0.        ,
        0.11952286],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.07905694, 0.        , 0.11952286, ..., 0.        , 0.        ,
        1.        ]])

In [537]:
index = df[df['names']=="Artemis Fowl"].index[0]

In [538]:
distances = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda vector:vector[1])
distances[:10]

[(392, 1.0),
 (548, 0.29250896965085227),
 (4, 0.269069117598525),
 (1435, 0.24253562503633297),
 (605, 0.2193817272381392),
 (1814, 0.21780342093451605),
 (1468, 0.2100420126042015),
 (1315, 0.20180183819889375),
 (1017, 0.19802950859533489),
 (518, 0.19446111706564934)]

In [539]:
for i in distances[:10]:
    print(df.iloc[i[0]].names)

Artemis Fowl
Jackass Presents: Bad Grandpa
Hatching
Home Team
Eight Crazy Nights
Little Boy
The Ridiculous 6
Billy Lynn's Long Halftime Walk
The Unheard
The Adam Project


In [540]:
def recommend(movies):
    index = df[df['names']==movies].index[0]
    distances = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda vector:vector[1])
    for i in distances[1:11]:
        print(df.iloc[i[0]].names)

In [541]:
start_column_index = 7
target_genre = 'Action'
movie_names = []
for movie in range(len(df)):
    movie_row = df.iloc[movie, start_column_index:]
    if movie_row[target_genre] and not movie_row[movie_row.index != target_genre].any():
        movie_names.append(df.iloc[movie]['names'])


if len(movie_names) <= 10:
    {print(movie) for movie in movie_names}
    recommend(movie_names[0])
else:
    {print(movie) for movie in movie_names[:15]}



The Sorcerer's Apprentice
Stop! Or My Mom Will Shoot
The Hard Way
Project A
Outlander
Spider-Man
Wu Kong
Battle for the Planet of the Apes
A Good Man
Avengers: Infinity War
The War of the Worlds
Never Back Down: Revolt
This Is the End
Bumblebee
Aquaman


In [542]:
recommend("1408")

The Machinist
Four Rooms
The Night Clerk
Hotel for Dogs
The Boy Next Door
A Hard Day
Hell House LLC II: The Abaddon Hotel
Beast
Grand Isle
They Live Inside Us


In [543]:
df.head()

,names,overview,TV_Movie,Adventure,Comedy,Fantasy,Romance,Family,History,Documentary,...,Crime,Drama,Horror,Thriller,Animation,Action,Science_Fiction,Mystery,Western,Music
0,The Sorcerer's Apprentice,Balthazar Blake is a master sorcerer in modern...,0,1,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,With Aunt It's Not a Sin,"The rich Gloria, resident for years in the Uni...",0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Being John Malkovich,"One day at work, unsuccessful puppeteer Craig ...",0,0,1,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,Oz the Great and Powerful,"Oscar Diggs, a small-time circus illusionist a...",0,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Hatching,12 year old Tinja is desperate to please her m...,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [544]:
# df["country"].unique()

In [545]:
# # Mapping dictionary
# country_mapping = {
#     'AU': 'Australia',
#     'IT': 'Italy',
#     'HK': 'Hong Kong',
#     'US': 'United States',
#     'ES': 'Spain',
#     'PL': 'Poland',
#     'DE': 'Germany',
#     'MX': 'Mexico',
#     'FR': 'France',
#     'JP': 'Japan',
#     'KR': 'South Korea',
#     'TR': 'Turkey',
#     'GB': 'United Kingdom',
#     'CA': 'Canada',
#     'CN': 'China',
#     'CO': 'Colombia',
#     'XC': 'Christmas Island',
#     'UA': 'Ukraine',
#     'AR': 'Argentina',
#     'TH': 'Thailand',
#     'NO': 'Norway',
#     'PH': 'Philippines',
#     'SU': 'Soviet Union',
#     'TW': 'Taiwan',
#     'BR': 'Brazil',
#     'RU': 'Russia',
#     'FI': 'Finland',
#     'DK': 'Denmark',
#     'IN': 'India',
#     'PE': 'Peru',
#     'ID': 'Indonesia',
#     'SG': 'Singapore',
#     'NL': 'Netherlands',
#     'SE': 'Sweden',
#     'KH': 'Cambodia',
#     'GR': 'Greece',
#     'BE': 'Belgium',
#     'CH': 'Switzerland',
#     'VN': 'Vietnam',
#     'CL': 'Chile',
#     'AT': 'Austria',
#     'UY': 'Uruguay'
# }

# # Replace country codes with full forms
# df['country'] = df['country'].map(country_mapping)


In [546]:
# df.head()

In [547]:
import pickle
pickle.dump(df, open("imdb_data.pkl", "wb"))

In [548]:
pickle.load(open("imdb_data.pkl", "rb"))

,names,overview,TV_Movie,Adventure,Comedy,Fantasy,Romance,Family,History,Documentary,...,Crime,Drama,Horror,Thriller,Animation,Action,Science_Fiction,Mystery,Western,Music
0,The Sorcerer's Apprentice,Balthazar Blake is a master sorcerer in modern...,0,1,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,With Aunt It's Not a Sin,"The rich Gloria, resident for years in the Uni...",0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Being John Malkovich,"One day at work, unsuccessful puppeteer Craig ...",0,0,1,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,Oz the Great and Powerful,"Oscar Diggs, a small-time circus illusionist a...",0,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Hatching,12 year old Tinja is desperate to please her m...,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019,Devil's Workshop,"Desperate for a role as a demonologist, strugg...",0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2020,The Dark Crystal,"On another planet in the distant past, a Gelfl...",0,1,0,1,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
2021,Pokémon the Movie: I Choose You!,Ash Ketchum wakes up late one morning after ha...,0,1,0,1,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
2022,The Curse of La Llorona,A social worker dealing with the disappearance...,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,1,0,0


In [549]:
df.columns

Index(['names', 'overview', 'TV_Movie', 'Adventure', 'Comedy', 'Fantasy',
       'Romance', 'Family', 'History', 'Documentary', 'War', 'Crime', 'Drama',
       'Horror', 'Thriller', 'Animation', 'Action', 'Science_Fiction',
       'Mystery', 'Western', 'Music'],
      dtype='object')

In [552]:
# q1 = df['score'].quantile(0.25)
# q3 = df['score'].quantile(0.75)
# IQR = q3 - q1
# IQR

In [553]:
df.to_csv("updated_imdb.csv", index=False)